In [1]:
from dataset import SiameseGoogleFer

In [2]:
import time
from torchvision import transforms

t = time.time()
train_path = "data/faceexp-comparison-data-train-public.csv"
test_path = "data/faceexp-comparison-data-test-public.csv"
transform = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])
train_dataset = SiameseGoogleFer(train_path, train_flag=True, divisions=5000, transform=transform)
test_dataset = SiameseGoogleFer(test_path, train_flag=False, divisions=5000, transform=transform)
print(time.time()-t)

89
10
2.5702099800109863


In [4]:
print(train_dataset.__len__())
test_dataset.__len__()
import torch 
from torch.autograd import Variable
batch_size = 4

from tripletnet import FECNet, EmbeddNet
model = EmbeddNet()
tnet = FECNet(model)
kwargs = {}
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)

for batch_idx, (data1, data2, data3) in enumerate(train_loader):
    data1, data2, data3 = Variable(data1), Variable(data2), Variable(data3)
    dista, distb, distc, embedded_x, embedded_y, embedded_z = tnet(data1, data2, data3)
    break
embedded_x.size()    

69


torch.Size([4, 16])

In [19]:
from densenet import *
class DenseNet(nn.Module):
    def __init__(self, growthRate, depth, input_channels, reduction, nClasses, bottleneck):
        super(DenseNet, self).__init__()

        nDenseBlocks = (depth-4) // 3
        if bottleneck:
            nDenseBlocks //= 2

        nChannels = 2*growthRate
        self.conv1 = nn.Conv2d(input_channels, nChannels, kernel_size=3, padding=1,
                               bias=False)
        self.dense1 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans1 = Transition(nChannels, nOutChannels)

        nChannels = nOutChannels
        self.dense2 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans2 = Transition(nChannels, nOutChannels)

        nChannels = nOutChannels
        self.dense3 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate

        self.bn1 = nn.BatchNorm2d(nChannels)
        #self.fc = nn.Linear(128, nClasses)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()

    def _make_dense(self, nChannels, growthRate, nDenseBlocks, bottleneck):
        layers = []
        for i in range(int(nDenseBlocks)):
            if bottleneck:
                layers.append(Bottleneck(nChannels, growthRate))
            else:
                layers.append(SingleLayer(nChannels, growthRate))
            nChannels += growthRate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.dense3(out)
        print(out.size())
        # print(out.data.shape)
        out = F.relu(self.bn1(out))
        print(out.size())
        #out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        # print(out.data.shape)
        #out = F.sigmoid(self.fc(out))
        return out

In [22]:
import torch.nn as nn
import torch.nn.functional as F
from facenet_pytorch import InceptionResnetV1
comp_resnet_pretrained = InceptionResnetV1(pretrained='vggface2')
modules = list(comp_resnet_pretrained.children())[:-8]
resnet = nn.Sequential(*modules)
densenet = DenseNet(growthRate=64, depth=5, input_channels=896, reduction=0.5, bottleneck=True, nClasses=1)

In [23]:
densenet(resnet(data1)).size()

torch.Size([4, 32, 3, 3])
torch.Size([4, 32, 3, 3])


torch.Size([4, 288])

torch.Size([4, 1792, 5, 5])

In [28]:
from dataset import *
from train import *

ModuleNotFoundError: No module named 'train'

In [ ]:
run train_and_test.py --epochs=10

<Response [410]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
  + Number of params: 27918535
loss  after  1  epoch:  1.4227095246315002
test accuracy after  1  epoch:  32.142857142857146
loss  after  2  epoch:  1.3273468017578125
test accuracy after  2  epoch:  35.714285714285715
loss  after  3  epoch:  1.1701658964157104
test accuracy after  3  epoch:  32.142857142857146


In [3]:
acc

NameError: name 'acc' is not defined

In [11]:
from facenet_pytorch import MTCNN, InceptionResnetV1
resnet = InceptionResnetV1(pretrained='vggface2')

In [7]:
from torchvision import transforms
import torch
kwargs = {'num_workers': 1, 'pin_memory': True} if False else {}
batch_size=4
transform = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])
train_path = "data/faceexp-comparison-data-train-public.csv"
test_path = "data/faceexp-comparison-data-test-public.csv"
train_dataset = SiameseGoogleFer(train_path, train_flag=True, transform=transform)
test_dataset = SiameseGoogleFer(test_path, train_flag=False, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

<Response [410]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>


In [12]:
from tripletnet import FECNet, EmbeddNet
model = EmbeddNet()
tnet = FECNet(model)
for batch_idx, (data1, data2, data3) in enumerate(train_loader):
    data1, data2, data3 = Variable(data1), Variable(data2), Variable(data3)
    dista, distb, distc, embedded_x, embedded_y, embedded_z = tnet(data1, data2, data3)
    

In [14]:
embedded_x

tensor([[-0.0119,  0.0215,  0.0509, -0.0129, -0.0075, -0.0220,  0.0671, -0.0386,
          0.0093, -0.0005, -0.0224,  0.0014, -0.0047, -0.0448, -0.0154, -0.0338],
        [ 0.0093,  0.0433,  0.0337, -0.0034,  0.0161, -0.0100,  0.0040,  0.0010,
          0.0424, -0.0313, -0.0732, -0.0305, -0.0524, -0.0471, -0.0571, -0.0421],
        [ 0.0280,  0.0102,  0.0265,  0.0034, -0.0402,  0.0233, -0.0097,  0.0020,
         -0.0412, -0.0133, -0.0102, -0.0455, -0.0431, -0.0025, -0.0458, -0.0441],
        [ 0.0355,  0.0185, -0.0030, -0.0201,  0.0118,  0.0008, -0.0089, -0.0357,
         -0.0539, -0.0302, -0.0295, -0.0163, -0.0457, -0.0379, -0.0565, -0.0340]],
       grad_fn=<AddmmBackward>)

In [23]:
from losses import TripletLoss

In [25]:
criterion = TripletLoss(margin=0.2)

In [27]:
criterion(embedded_x, embedded_y, embedded_z)

tensor(0.2055, grad_fn=<MeanBackward0>)

In [44]:
import numpy
def triplet_accuracy(dista, distb, distc):
    return numpy.logical_and(dista < distb, dista < distc).sum()

In [45]:
triplet_accuracy(dista.detach().numpy(), distb.detach().numpy(), distc.detach().numpy())

2

In [32]:
dista

tensor([0.1345, 0.1593, 0.1257, 0.1172], grad_fn=<NormBackward1>)

In [46]:
distb

tensor([0.0899, 0.0826, 0.1411, 0.1278], grad_fn=<NormBackward1>)

In [47]:
distc

tensor([0.1061, 0.1316, 0.1354, 0.1413], grad_fn=<NormBackward1>)

In [51]:
cnt = 0
for parama in tnet.parameters():
    cnt = cnt+1

In [52]:
cnt

382

In [54]:
tnet.embeddingnet.resnet.requires_grad = False